In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.Qiskit_CP_VQA import CP_VQA


from src.Chain import Chain
from src.Qubo import Qubo
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       normalized_cost,
                       get_qubo, 
                       qubo_limits, 
                       check_qubo)

In [ ]:
N_min, N_max = 4, 4

qiskit_full_res = {N: [] for N in range(N_min, N_max + 1)}
qiskit_reduced_res = {N: [] for N in range(N_min, N_max + 1)}


max_iter = 1000
N_seeds = 5
start_seed = 100
w_nnn = False

layer_dict = {4:  2, 5:  1, 6 : 1,
              7:  2, 8 : 2, 9 : 2, 
              10: 3, 11: 3, 12: 3, 
              13: 4, 14: 4, 15: 4, 
              16: 5, 17: 5, 18: 5, 
              19: 6, 20: 6, 21: 6}
 # Choosing optimizer for scipy
optimizer_method = 'COBYLA'
for N in tqdm(range(N_min, N_max+1)):
    k = N // 2
    init_strat = np.array([1 if i%2 == 1 else 0 for i in range(N)])
    my_indices = [(i, i+1) for i in range(N-1)]
    alpha = 0.5
    for repetition in range(N_seeds):
         # Defining topology
        my_chain = Chain(N_qubits=N)
        my_chain.set_initialization_strategy(strategy=init_strat)
        # Deciding between grid and 1d chain topology
        my_topology = my_chain
        # Generating random problem instance 
        expected_returns, covariances = portfolio_metrics(n=N, seed=start_seed)
        # Retrieving C_min, C_max and corresponding states for original portfolio problem
        constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=N,
                                                                    k=k,
                                                                    mu=expected_returns,
                                                                    sigma=covariances,
                                                                    alpha=alpha)
        portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
        portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])
        # Generating QUBO corresponding to current problem instance
        Q, offset = get_qubo(mu=expected_returns,
                             sigma=covariances, 
                             alpha=alpha,
                             lmbda=lmbda+1e-8, # Adding small constant purposely
                             k=k)
        qubo = Qubo(Q=Q, offset=offset) 
        QUBO_limits = qubo_limits(Q=Q,offset=offset)
        qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
        qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
        check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
        qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])
        
        if not portfolio_subspace_min_state_str == qubo_min_state_str:
            raise RuntimeError(f'portfolio_subspace_min_state_str: {portfolio_subspace_min_state_str}, qubo_min_state_str={qubo_min_state_str}'+f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')
        
        if not np.isclose(qubo_min_cost,portfolio_subspace_min_cost):
            raise RuntimeError(f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')
        
        if not qubo_max_cost >= portfolio_subspace_max_cost:
            raise RuntimeError(f'Max. cost of qubo: {qubo_max_cost}, max. cost of portfolio subspace: {portfolio_subspace_max_cost} (should be qubo max. >= constrained portfolio max)')
         
        qiskit_ansatz_full = CP_VQA(N_qubits=N,
                                    cardinality=k,
                                    layers=layer_dict[N],
                                    topology=my_topology,
                                    with_next_nearest_neighbors=w_nnn,
                                    approximate_hamiltonian=False,
                                    qubo=qubo)
         
        qiskit_ansatz_reduced = CP_VQA(N_qubits=N,
                                        cardinality=k,
                                        layers=layer_dict[N],
                                        topology=my_topology,
                                        with_next_nearest_neighbors=w_nnn,
                                        approximate_hamiltonian=True,
                                        qubo=qubo)
         
        
         # Generating initial guess for rotation angles
        np.random.seed(start_seed)
        theta_min, theta_max = -2*np.pi, 2*np.pi
        N_angles = layer_dict[N] * len(my_topology.get_NNN_indices()) if w_nnn else layer_dict[N] * len(my_topology.get_NN_indices())
        theta_i = np.random.uniform(-2*np.pi, 2*np.pi, N_angles)
        
        qiskit_sim_full = sc.optimize.minimize(fun=qiskit_ansatz_full.get_cost, 
                                               x0=theta_i,
                                               method=optimizer_method,
                                               options={'disp': False, 
                                                        'maxiter': max_iter})
         
        full_norm_c = normalized_cost(result=qiskit_ansatz_full.counts,
                                      QUBO_matrix=Q,
                                      QUBO_offset=offset,
                                      max_cost=portfolio_subspace_max_cost, 
                                      min_cost=qubo_min_cost)
        if portfolio_subspace_min_state_str in list(qiskit_ansatz_full.counts.keys()):
            full_norm_p = qiskit_ansatz_full.counts[portfolio_subspace_min_state_str]
        else:
            full_norm_p = 0.0
        qiskit_full_res[N].append([full_norm_c,full_norm_p])     
        
        
        qiskit_sim_reduced = sc.optimize.minimize(fun=qiskit_ansatz_reduced.get_cost, 
                                                  x0=theta_i,
                                                  method=optimizer_method,
                                                  options={'disp': False, 
                                                            'maxiter': max_iter})
        reduced_norm_c = normalized_cost(result=qiskit_ansatz_reduced.counts,
                                         QUBO_matrix=Q,
                                         QUBO_offset=offset,
                                         max_cost=portfolio_subspace_max_cost, 
                                         min_cost=qubo_min_cost)
        if portfolio_subspace_min_state_str in list(qiskit_ansatz_reduced.counts.keys()):
            reduced_norm_p = qiskit_ansatz_reduced.counts[portfolio_subspace_min_state_str]
        else:
            reduced_norm_p = 0.0
        qiskit_reduced_res[N].append([reduced_norm_c,reduced_norm_p])     
        
        start_seed += 1

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
qiskit_full_res

{4: [[0.0, 0.554871551184684],
  [0.0, 0.999999953816479],
  [0.008983914509257956, 2.256687e-09],
  [0.2937125184392704, 7.02971e-10],
  [0.0, 0.99986757078312]]}

In [6]:
qiskit_reduced_res

{4: [[0.18113500082497608, 0.0],
  [0.0, 0.999999996201461],
  [0.008983914509257956, 0.0],
  [0.2937125184392704, 0.0],
  [0.0, 0.999999986748989]]}